In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import datetime
import os

def load_flight_info(date, month):
    print(date)
    year = 2023
    date_month = str(month) + "/" + str(date)
    if driver.current_url == "https://www.koreanair.com/booking/select-flight/departure":
        flight_date = date_month
        flight_time = driver.find_elements(By.CLASS_NAME, "flight__time")
        flight_duration = driver.find_elements(By.CLASS_NAME, "flight__duration")
        flight_transfer = driver.find_elements(By.CLASS_NAME, "flight__stop")
        flight_number = driver.find_elements(By.CLASS_NAME, "flight__number")
        flight_fares = driver.find_elements(By.CLASS_NAME, "flight__fare")
        today = datetime.date.today()
        if month == 11 or month == 12:
            year = 2022
        time.sleep(0.5)
        target_day = datetime.date(year, month, int(date))
        d_day = target_day - today
        
        i = 1
        while i != len(flight_fares)+1:
            try:                                                
                flight__transfer = driver.find_element(By.XPATH, "/html/body/app-root/div/ke-selection-flight/ke-basic-layout/div[1]/div/div[2]/div[2]/ke-air-offer-bounds-cont/ke-air-offer-bounds-pres/div/div[2]/ul/li[{}]/div[1]/a/div[1]/div/div[3]/div[1]/ke-flight-desc-item/span[3]".format(i))
                transfer_list.append("O")
                i += 1 
            except:
                transfer_list.append("X")
                i += 1 
                
        transfer_index = 0
        transfer_index2 = 0
        
        for i, item in enumerate(transfer_list):
            if item == "O":
                data_dic['date'].append(flight_date)
                data_dic['duration'].append(flight_duration[i].text)
                data_dic['price'].append(flight_fares[i].text)
                data_dic['time1'].append(flight_time[i].text)
                data_dic['time2'].append(flight_time[i+1].text)
                data_dic['left_date'].append(d_day.days)
            
                data_dic['transfer'].append(flight_transfer[transfer_index2].text)
                data_dic['flight_number1'].append(flight_number[transfer_index].text)
                data_dic['flight_number2'].append(flight_number[transfer_index+1].text)
                transfer_index += 2 
                transfer_index2 += 1 
            
            else:
                data_dic['date'].append(flight_date)
                data_dic['duration'].append(flight_duration[i].text)
                data_dic['price'].append(flight_fares[i].text)
                data_dic['time1'].append(flight_time[i].text)
                data_dic['time2'].append(flight_time[i+1].text)
                data_dic['left_date'].append(d_day.days)
            
                data_dic['transfer'].append("")
                data_dic['flight_number1'].append(flight_number[transfer_index].text)
                data_dic['flight_number2'].append("")
                transfer_index += 1
                
        driver.back()
    else:
        driver.back()
        time.sleep(0.5)
        driver.back()

def reset_data(data_dic, transfer_list):
    transfer_list = []
    data_dic = {'date' : [],
            'time1' : [],
            'time2' : [],
            'duration' : [],
            'transfer' : [],
            'flight_number1' : [],
            'flight_number2' : [],
            "left_date" : [],
            'price' : []}
    return data_dic, transfer_list

def to_csv(data_dic):
    data_dic2 = pd.DataFrame(data_dic)
    if os.path.exists('newyork.csv'):
        data_dic2.to_csv("newyork.csv", header=False, index=False, mode="a", encoding="utf-8-sig")
    else:
        data_dic2.to_csv("newyork.csv", index=False, encoding="utf-8-sig")
    
url = "https://www.koreanair.com/booking/select-flight/departure"
destination_list = ["뉴욕", "도쿄"]
transfer_list = []
date = "1"
data_dic = {'date' : [],
            'time1' : [],
            'time2' : [],
            'duration' : [],
            'transfer' : [],
            'flight_number1' : [],
            'flight_number2' : [],
            "left_date" : [],
            'price' : []}

month_list = [12, 1, 2, 3, 4, 5]
month_dict = {12 : 32, 1 : 32, 2 : 29, 3 : 32, 4 : 31, 5 : 32}
count = 1

driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(10)

for j in destination_list:
    for i in month_list:
        while date != str(month_dict[i]):
            start_time = time.time()
            destination_btn = driver.find_elements(By.CLASS_NAME, 'booking-new__button')
            destination_btn[1].click()
            destination_name = driver.find_element(By.CLASS_NAME, 'auto-search__text')
            destination_name.send_keys(j)
            time.sleep(1)
            destination_name.send_keys(Keys.ARROW_DOWN)
            destination_name.send_keys(Keys.ENTER)
            one_way_btn = driver.find_elements(By.CLASS_NAME, 'selection')
            one_way_btn[1].click()
            date_btn = driver.find_element(By.CLASS_NAME, "booking-new__datepicker")
            date_btn.click()
            flight_date = driver.find_elements(By.XPATH, "//span[text()='{}']".format(date))
            flight_date[count].click()
            confirm_btn = driver.find_element(By.XPATH, "//button[contains(text(),'확인')]")
            confirm_btn.click()
            time.sleep(0.5)
            confirm_btn2 = driver.find_element(By.CLASS_NAME, "booking-new__search")
            confirm_btn2.click()
            driver.implicitly_wait(10)
            time.sleep(2)
    
            load_flight_info(date, i)
            to_csv(data_dic)
            data_dic, transfer_list = reset_data(data_dic, transfer_list)
    
            time.sleep(0.5)
            date = int(date) + 1
            date = str(date)
            end_time = time.time()
            print(end_time - start_time)
        
        count += 1
        date = "1"
        print("루프 종료", i)
    count = 1
    print("루프 종료", j)